In [1]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [2]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)

In [3]:
doc._.polarity

-0.125

In [4]:
doc._.subjectivity

0.9

In [5]:
text = "I'm very happy. I'm working on a beautiful project."
doc = nlp(text)

In [6]:
doc._.polarity

0.925

In [7]:
doc._.subjectivity

1.0

In [8]:
text = "This book is very simple and enjoyable to study."
doc = nlp(text)

In [9]:
doc._.polarity

0.25

In [10]:
doc._.subjectivity

0.5321428571428571

# CONFRONTIAMO IL NOSTRO MODELLO NAIVE BAYES CON SPACY

### - Caricare il dataset SMALL_IMDB.csv
### - Splittare il dataset in train e test
### - Pulire i dataset di train e test (salvare il dataset di test non pulitò, servirà con spacy)
### - Fare il training del modello Naive Bayes
### - Testare il modello sul dataset di test
### - Utilizzare la funzione polarity di spacy per calcolare la sentiment (scrivere opportuna funzione che associa al valore numerico le label positive e negative)
### - Scrivere una funzione per calcolare l'accuracy ottenuta dai risultati spacy

## Puliamo il dataset, addestriamo il modello e valutiamo l'accuracy

In [45]:
import pandas as pd

In [46]:
dataset = pd.read_csv('datasets/Lezione_5-sentiment_Analysis/SMALL_IMDB.csv')

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
data_train, data_test, y_train, y_test = train_test_split(dataset['review'], dataset['sentiment'], test_size=0.20, random_state=40)

In [49]:
import string
import spacy
from nltk.corpus import stopwords
import re

english_stopwords = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
punctuation = set(string.punctuation)

def data_cleaner(sentence):
    sentence = sentence.lower()
    for c in string.punctuation:
        sentence = sentence.replace(c, " ")
    document = nlp(sentence)
    sentence = ' '.join(token.lemma_ for token in document)
    sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords)
    sentence = re.sub('\d', '', sentence)
    
    return sentence

In [50]:
data_train_cleaned = []
for i,r in enumerate(data_train):
    data_train_cleaned.append(data_cleaner(r))

In [51]:
data_test_cleaned = []
for r in data_test:
    data_test_cleaned.append(data_cleaner(r))

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
data_train_cleaned_vectorized = vec.fit_transform(data_train_cleaned).toarray()
data_test_cleaned_vectorized = vec.transform(data_test_cleaned).toarray()

In [53]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(data_train_cleaned_vectorized, y_train)

MultinomialNB()

In [54]:
model.score(data_test_cleaned_vectorized, y_test)

0.832

## Valutiamo Spacy 

In [80]:
def sentiment_checker(value, label, text):
    if (value > 0 and label == 'positive') or (value < 0 and label == 'negative'):
        return 1
    else:
        #print(text)
        print("POLARITY SPACY: "+str(value))
        print("REAL LABEL: "+label)
        print("OUR SA MODEL: "+model.predict(vec.transform([data_cleaner(text)]))[0])
        print("----------------------------------------------------------------")
        return 0

In [81]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

count = 0
for r,s in zip(data_test,y_test):
    doc = nlp(r)
    count+=sentiment_checker(doc._.polarity,s,r)
    

I like all different types of movies, so this is not a bash on romantic movies from a guy who only likes The Matrix etc etc.<br /><br />I just felt it was a lousy movie. I don't feel that there was enough buildup of the characters to fall in love. They were there for a few days and while dealing with a severe hurricane and major issues in each of their lives, Richard Gere and Diane Lane fall hopelessly and helplessly in love?? It isn't realistic. This movie didn't make me buy into it and feel it emotionally and that is something that you look for in a good plot. Some emotional connection. If someone can relate to them falling in love that quickly, without any true substance , than so be it. You are neither right or wrong. Different strokes for different folks.<br /><br />Another very unbelievable component to the movie was Diane Lane's very rude teenage daughter becoming nice and sympathetic at the end of the movie. Does a teenage girl who is that miserable and aggravated at her mother

I watched this with great trepidation, and my trepidation was well founded, it seems. What was this movie about? Knight? The season? The Players? What? It was all over the place all the time. It had no tension (sorry, we all knew Bobby was going to curse and throw things) but Brian Denehy, a fine actor, comes across as mailing in the anger and delivering zero tension. Cheaply shot, like a MacGiver episode. Contrast this Thanksgiving main course with the job HBO did on the Don King movie "only in America" to show how to do sports biopics, warts and all. Notice that ESPN promoted the hell out of it and then never showed it again?
POLARITY SPACY: 0.08333333333333336
REAL LABEL: negative
OUR SA MODEL: positive
----------------------------------------------------------------
"I thought I'd be locked away in a padded cell and they'd throw away the key" (Thus is a paraphrased snatch of dialogue from "State of Mind".<br /><br />One wonders in what tangled forest Paula Milne and her co-writer f

OUR SA MODEL: positive
----------------------------------------------------------------
A woman in love with her husband (he's suicidal) decides to have a baby to save his life. She's been to a fertility clinic - as has the lover she takes - so both know how artificial insemination works; but, instead of using the method thousands of people use every year around the world (the $5 turkey baster), they engage in coitus. We also are to believe that although the immigrant is in love with his fiancée, he doesn't suggest the obvious alternative to intercourse. Further, even though this is a business arrangement, the first time she's with her sperm donor, she takes off all her clothes, as if it's a seduction. Plus, her husband doesn't notice when $30,000 goes missing from their bank accounts. Does all this seem to demand more willing suspension of disbelief than even most Hollywood fare? Far fetched on all counts.
POLARITY SPACY: 0.16363636363636364
REAL LABEL: negative
OUR SA MODEL: negative

There are other movies about boarding schools and the antics of the students and staff, but "The Belles of St. Trinian's" towers above them all! The plot has been thoroughly summarized by other posters, so I won't cover the same ground. I just want to say that it's a shame that it's FINALLY out on DVD, but in a format that can't be used in the U.S.! :-( <br /><br />Enjoy, fellow fans in New Zealand and Australia! And if anyone reading this has any pull in such matters, PLEASE help get it released on DVD with Region 1 encoding! Also, is it possible to be notified via e-mail when (I won't say "if") it is released on DVD in the United States? Thanks!
POLARITY SPACY: -0.05795454545454546
REAL LABEL: positive
OUR SA MODEL: positive
----------------------------------------------------------------
This film is harmless escapist fun. Something the recent Tomb Raider film lacked. I can't wait to get the DVD. How can people give this a low score and still go and see Titanic without a guilty cons

This movie did not give Mr. Bachchan justice. He is a great actor and I was very disappointed in the movie and there was not much plot to it. Matter a fact this is the first movie I have ever been disappointed in with him in it. It starts out with her coming to his home with his daughter and he is a photographer. He takes pictures of her in the garden has she's hosing herself down. He is a sixty year old man that has nothing to do but to take pictures of this girl. The movie makes no sense. The whole movie is about her chasing him around and her telling him how much she cares about him. Then his daughter falls and he has to take this girl around places so she is not bored and is daughter finds out. I just didn't think this movie was up to Bachchan's standards. He is better than this movie. I always pictured him as an upstanding person and then I seen him in this movie and I couldn't picture him in this movie. The movie didn't hold my interest at all. I couldn't wait until the movie was

...

What happened? What we have here is basically a solid and plausible premise and with a decent and talented cast, but somewhere the movie loses it. Actually, it never really got going. There was a little excitement when we find out that Angie is not really pregnant, then find out that she is after all, but that was it. Steve Martin, who is a very talented person and usually brings a lot to a movie, was dreadful and his entire character was not even close to being important to this movie, other than to make it longer. I really would have liked to see more interactions between the main characters, Kate and Angie, and maybe try not for a pure comedy, which unfortunately it was not, but maybe a drama with comedic elements. I think if the movie did this it could have been very funny since both actresses are quite funny in their own ways and sitting here I can think of numerous scenarios that would have been a riot.
POLARITY SPACY: 0.12097718253968255
REAL LABEL: negative
OUR SA MODEL: negati

Upon viewing Tobe Hooper's gem, Crocodile, in 2000, I developed a great interest in the college/crocodile niche of the exploitation/monster genre. I look forward to a wayward producer to follow up with several sequels to these delightful bonbons of camp goodness. If only Ed Wood could bring his subtle sense of flair and dignity to these remarkable scripts. With Ed writing the scripts, and a room full of monkees creating crocodile special effects on a computer, all we'd need would be a cast of crocky fodder with Russ Meyer breasts and Ren Hoek pectoral implants.<br /><br />While Tobe Hooper's crocky opus referenced his own movies, Blood Surf chose to dish out a bunch of aging themes from the chum bucket of other movies. See if you can look past the Revenge of the Nerds sequel sets to find the allusions/homages?/rip-offs to Jaws, Temple of Doom, Indiana Jones' Last Crusade, The Convent, Godzilla 2000, and any James Bond movie. Also, try to find the ready-for-tv fade where the editor gave

I am a usually a very generous voter on IMDb and don't bother commenting on movies I did not like, but this was just lame. I actually turned it off 15 minutes before finishing it, to watch "This Is It" (because my gf wanted to... I just chose the lesser of two evils).<br /><br />If you want to watch this movie: picture this film as a collection of worse-than-average "horror"-stories, like "scary short-stories" that you find in an issue of "Reader's Digest" in the waiting room of your dentist's.<br /><br />I did not expect anything particular terrifying or funny, I am not the "I want to see blood!"-type of person, but this "movie" is neither "horror" nor "comedy" nor entertaining in any other way.<br /><br />It's probably more scary/funny and entertaining to look at the movie-poster of "You've Got Mail" for 90 minutes while drinking chamomile tea.<br /><br />Conclusion: a "horror-comedy" for people between 4 and 7.
POLARITY SPACY: 0.004513888888888891
REAL LABEL: negative
OUR SA MODEL: 

I wish I could give this movie a zero. Cheesy effects and acting. The only reason to see this movie is so you can see how bad it is. Lets start with the kid who plays Brian. What a geek! I couldn't believe the mullet! Then there was the talking to himself. I guess they couldn't just have the movie be silent, but still. Of course they had to have him skinny-dipping too, not something I wanted to see. But Jared gave a great performance, compared to the special effects department. Everything from the bear to the crash was something I could do myself, and better. I seriously doubt that Gary Paulsen had anything to do with the production, seeing as the movie was not even called Hatchet. Finally, I do not think the writer had ever read the book, seeing as nothing was the same. I think the book was great, but this movie stunk like a smelly goat!
POLARITY SPACY: 0.06865079365079366
REAL LABEL: negative
OUR SA MODEL: negative
----------------------------------------------------------------
If t

POLARITY SPACY: 0.08358106528245418
REAL LABEL: negative
OUR SA MODEL: negative
----------------------------------------------------------------
A couple are split apart during a vacation. Early scenes lead us to believe Barney, Jeff Bridges' character, has kidnapped the girl in an unexplained but premeditated abduction. The film plays out some years later where, despite a new love interest, Jeff (Kiefer Sutherland) is determined to solve her disappearance.<br /><br />The storyline is definitely intriguing but there have been some really bad decisions made during the making of this film, culminating a pretty shoddy piece of work.<br /><br />OK here we go. The abduction scene - aside from two major continuity errors there is a point where Sandra Bullock and Kiefer Sutherland are made to perform a little signed-pet-love-ritual that's drowned in over-sentimentality to the point of nausea. We know they're in love; the "I do solemnly swear never to leave you" speech only moments prior was a

In [82]:
count/len(y_test)

0.68